<a href="https://colab.research.google.com/github/Joelss23/TFM/blob/main/AlgoritmoTFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO DE APRENDIZAJE AUTOMÁTICO PARA LA GESTIÓN DE CALIDAD Y PRODUCCION VINICOLA: IMPLEMENTACIÓN DE UN SISTEMA DE CALIDAD BASADO EN DATOS

## Resumen

## #1. Instalación y Carga de Librerías

###Instalación de Paquetes

In [ ]:
#Instalación de paquetes necesarios
!pip install --upgrade google-cloud-storage openpyxl scikit-learn imbalanced-learn seaborn
!pip install google-cloud-bigquery pandas

### Importación de Librerias

In [ ]:
#Importación de librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Librerías para Machine Learning y Preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree

#Librerías para balanceo
from imblearn.over_sampling import SMOTE

#Librerías para Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Librerías para Google Cloud
from google.colab import files
from google.cloud import storage
from google.cloud import bigquery
from google.auth import default
from google.auth.exceptions import DefaultCredentialsError

#Librerías del sistema
import os

## #2. Autenticación, Carga y Descarga de Datos Desde Google Cloud Storage (GCS)

### Validación de Clave de Acceso

In [ ]:
#Subir el archivo de claves JSON para la autenticación
print("Por favor, sube el archivo de clave JSON para la autenticación:")
uploaded = files.upload()
json_key_path = '/content/Clave-acceso-GCS.json'  #Ajustar el nombre de archivo en caso de ser necesario

#Establecer la variable de entorno para la autenticación
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_path

#Verificar autenticación e inicializar cliente de GCS
try:
    storage_client = storage.Client()
    print("\nAutenticación exitosa con Google Cloud.")
except DefaultCredentialsError as e:
    print(f"\nError de autenticación: {e}")

### Carga del Archivo (Datos) al bucket de GCS

In [ ]:
#Subir archivo desde la máquina local
print("Por favor, sube el archivo Excel desde la máquina local:")
uploaded = files.upload()
local_excel_filename = list(uploaded.keys())[0]

#Subir el archivo al bucket
bucket_name = 'sistema-calidad-vino-datos'  #Nombre del proyecto en GCS
blob_name = 'Dataset_Vinos_TFM.xlsx'        #Nombre que tendrá el archivo en GCS

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)

#Subir a GCS
blob.upload_from_filename(local_excel_filename)

print(f"\nArchivo '{local_excel_filename}' subido exitosamente a gs://{bucket_name}/{blob_name}")

### Carga de Datos Interna

In [ ]:
#Guardar el DataFrame localmente como CSV
df = pd.read_excel(local_excel_filename)
csv_file_name = local_excel_filename.replace('.xlsx', '.csv')
df.to_csv(csv_file_name, index=False)

#Confirmación de la exportación
print(f"Archivo convertido a CSV y guardado como: {csv_file_name}")

## #3. Carga del Dataset y Validación Inicial

In [ ]:
#Cargar el dataset
#(Si se cambio el nombre del archivo previamente aca debera tener el mismo de arriba)
df = pd.read_excel('Dataset_Vinos_TFM.xlsx')

#Mostrar los primeros 5 y los últimos 5 registros para validar que se ha cargado correctamente
print("Primeros 5 registros:")
print(df.head())
print("\nÚltimos 5 registros:")
print(df.tail())

## #4. Analisis Exploratorio

### Informacion General del Dataset

In [ ]:
#Mostrar información general sobre las columnas (tipos de datos)
print("Información del dataset:")
print(df.info())

### Estadísticas Básicas del Dataset

In [ ]:
#Estadísticas básicas
print("Estadísticas básicas:")
print(df.describe())

### Información General de las Columnas del Dataset

In [ ]:
#Información general de las columnas
print("Información del dataset:")
print(df.info())

### Valores Nulos

In [ ]:
#Valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

### Filas Duplicadas

In [ ]:
#Filas duplicadas
print("Cantidad de filas duplicadas:")
print(df.duplicated().sum())

### Distrubición de Variables

In [ ]:
#Distribución de variables categóricas relevantes
print("Distribución por Tipo de Vino:")
print(df['Tipo de Vino'].value_counts())
print("\nDistribución por Tipo de Uva:")
print(df['Tipo de Uva'].value_counts())

## #5. Ingeniería de Variables - Crear Variable de Calidad

In [ ]:
#Función para evaluar calidad
def evaluar_calidad(row, tipo_vino):
    if tipo_vino == "VINOTINTO":
        return 'Buena' if (12.5 <= row['Grado Alcoholico Adquirido'] <= 14.5 and
                           20 <= row['Sulfuroso Libre (mg/L)'] <= 30 and
                           row['Sulfuroso Total (mg/L)'] <= 150 and
                           4.5 <= row['Acidez Total (g/L)'] <= 6.0 and
                           row['Acidez Volatil (g/L)'] <= 0.6 and
                           3.3 <= row['pH'] <= 3.6 and
                           row['Azucares Reductores (g/L)'] < 4) else 'Mala'
    elif tipo_vino == "VINOBLANCO":
        return 'Buena' if (11.5 <= row['Grado Alcoholico Adquirido'] <= 13.5 and
                           25 <= row['Sulfuroso Libre (mg/L)'] <= 40 and
                           row['Sulfuroso Total (mg/L)'] <= 180 and
                           5.5 <= row['Acidez Total (g/L)'] <= 7.5 and
                           row['Acidez Volatil (g/L)'] <= 0.5 and
                           3.1 <= row['pH'] <= 3.4 and
                           row['Azucares Reductores (g/L)'] < 4) else 'Mala'
    elif tipo_vino == "VINOROSADO":
        return 'Buena' if (11.5 <= row['Grado Alcoholico Adquirido'] <= 13.5 and
                           25 <= row['Sulfuroso Libre (mg/L)'] <= 40 and
                           row['Sulfuroso Total (mg/L)'] <= 180 and
                           5.0 <= row['Acidez Total (g/L)'] <= 7.0 and
                           row['Acidez Volatil (g/L)'] <= 0.5 and
                           3.1 <= row['pH'] <= 3.5 and
                           row['Azucares Reductores (g/L)'] < 4) else 'Mala'
    return 'Desconocido'

#Aplicar función
df['Conformidad'] = df.apply(lambda row: evaluar_calidad(row, row['Tipo de Vino']), axis=1)

#Visualizar nuevas etiquetas
print("\nDistribución de la variable 'Conformidad':")
print(df['Conformidad'].value_counts())

## #6. Limpieza de Datos

### Limpieza y Preparación de Datos

In [ ]:
#Eliminar columnas no numéricas innecesarias
df_cleaned = df.drop(columns=['# Muestra'])

#Separar las columnas numéricas (excluir las categóricas)
columns_numericas = df_cleaned.select_dtypes(include=[np.number]).columns
X = df_cleaned[columns_numericas]  # Sólo las columnas numéricas
y = df_cleaned['Conformidad']

#Codificar variable objetivo a 0 (Mala) y 1 (Buena)
df_cleaned = df_cleaned[df_cleaned['Conformidad'].isin(['Buena', 'Mala'])]
df_cleaned['Conformidad_Bin'] = df_cleaned['Conformidad'].map({'Mala': 0, 'Buena': 1})

### División de Datos y Normalización

In [ ]:
#Preparar datos
X = df_cleaned.select_dtypes(include=[np.number]).drop(columns=['Conformidad_Bin'])
y = df_cleaned['Conformidad_Bin']

#Dividir el dataset en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Normalizar solo las columnas numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[columns_numericas])

#Normalizar conjunto de prueba (utilizando el mismo escalador que se entrenó con X_train)
X_test_scaled = scaler.transform(X_test[columns_numericas])

### Conversión a DataFrame y Verificación

In [ ]:
#Convertir los datos escalados nuevamente a DataFrame para facilitar su manejo
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=columns_numericas)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=columns_numericas)

#Verificar la forma de los datos escalados
print(X_train_scaled_df.head())
print(X_test_scaled_df.head())

## #7.  Análisis Separado por Tipo de Vino y Tipo de Uva

### #7.1 Análisis por Tipo de Vino

In [ ]:
#Dividir el dataset en subconjuntos por tipo de vino
vinotinto = df_cleaned[df_cleaned['Tipo de Vino'] == 'VINOTINTO']
vinoblanco = df_cleaned[df_cleaned['Tipo de Vino'] == 'VINOBLANCO']
vinorosado = df_cleaned[df_cleaned['Tipo de Vino'] == 'VINOROSADO']

#### Vinotinto

In [ ]:
print("Análisis para Vino Tinto:")
print(vinotinto.describe())

#### Vino Blanco

In [ ]:
print("Análisis para Vino Blanco:")
print(vinoblanco.describe())

#### Vino Rosado

In [ ]:
print("Análisis para Vino Rosado:")
print(vinorosado.describe())

### #7.2 Analisis por Tipo de Uva (Vinotinto)

In [ ]:
#Dividir el dataset en subconjuntos por tipo de uva
syrah = df_cleaned[df_cleaned['Tipo de Uva'] == 'SYRAH']
graciano = df_cleaned[df_cleaned['Tipo de Uva'] == 'GRACIANO']
cabernet_sauvignon = df_cleaned[df_cleaned['Tipo de Uva'] == 'CABERNET SAUVIGNON']
sauvignon_blanco = df_cleaned[df_cleaned['Tipo de Uva'] == 'SAUVIGNON BLANCO']
rosado = df_cleaned[df_cleaned['Tipo de Uva'] == 'ROSADO']

#### Syrah

In [ ]:
print("Análisis para uva Syrah:")
print(syrah.describe())

#### Graciano

In [ ]:
print("Análisis para uva Graciano:")
print(graciano.describe())

#### Cabernet Sauvignon

In [ ]:
print("Análisis para uva Cabernet Sauvignon:")
print(cabernet_sauvignon.describe())

## #8. Modelado con TensorFlow

### Creación del Modelo y Entrenamiento

In [ ]:
#Crear el modelo de red neuronal de TensorFlow
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Entrenar el modelo
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

### Predicciones

In [ ]:
#Realizar predicciones
nn_probs = model.predict(X_test_scaled).flatten()  # predicciones probabilísticas
nn_preds = (nn_probs > 0.5).astype(int)  # convertir a clases 0 o 1

### Evaluación del Modelo

In [ ]:
#Evaluar el desempeño de la red neuronal
print("Modelo: Red Neuronal")
print(classification_report(y_test, nn_preds))

## #9. Análisis y Evaluación del Modelo Predictivo

### #9.1 Analisis de Outliers

In [ ]:
#Análisis de outliers
sns.boxplot(data=X)
plt.xticks(rotation=90)
plt.title('Distribución de variables - Outliers')
plt.show()

### #9.2 Balance de clases con SMOTE

In [ ]:
#Balance de clases con SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)
print("Distribución de clases después de aplicar SMOTE:")
print(pd.Series(y_resampled).value_counts())

### #9.3 Validación Cruzada

In [ ]:
#Validación cruzada
cross_val_scores = cross_val_score(RandomForestClassifier(), X_resampled, y_resampled, cv=5)
print(f"Puntuaciones de validación cruzada: {cross_val_scores}")
print(f"Precisión media de validación cruzada: {cross_val_scores.mean():.4f}")

## #10. Comparación de Modelos

### #10.1 Random Forest

In [ ]:
#Diccionario para almacenar las precisiones de cada modelo
model_accuracies = {}

#Entrenar y evaluar el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

#Realizar predicciones
rf_preds = rf_model.predict(X_test_scaled)

#Almacenar precisión
model_accuracies["Random Forest"] = accuracy_score(y_test, rf_preds)

#Visualización del Random Forest
estimator = rf_model.estimators_[0]
plt.figure(figsize=(30, 10))
plot_tree(estimator, filled=True, feature_names=X_train.columns, class_names=['Mala', 'Buena'], max_depth=3)
plt.title("Visualización del Random Forest")
plt.show()

### #10.2 Regresión Logística

In [ ]:
#Entrenar y evaluar el modelo de Regresión Logística
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

#Realizar predicciones
lr_preds = lr_model.predict(X_test_scaled)

#Almacenar precisión
model_accuracies["Regresion Logistica"] = accuracy_score(y_test, lr_preds)

#Obtener coeficientes de la regresión logística
coeficientes = lr_model.coef_[0]
variables = X_train.columns

#Crear un nuevo DataFrame con los coeficientes
coef_df = pd.DataFrame({'Variable': variables, 'Coeficiente': coeficientes})
coef_df = coef_df.sort_values(by='Coeficiente')

#Visualización de la importancia de las variables
plt.figure(figsize=(10, 6))
sns.barplot(x='Coeficiente', y='Variable', data=coef_df, palette="coolwarm")
plt.title("Importancia de las Variables - Regresión Logística")
plt.axvline(0, color='black', linestyle='--')
plt.show()

### #10.3 Comparación de Modelos

#### Entrenamiento y Evaluación de Modelos

In [ ]:
#Comparar Red Neuronal, Random Forest y Regresión Logística
models = {
    "Red Neuronal": model,
    "Random Forest": rf_model,
    "Regresion Logistica": lr_model
}

#Entrenar y evaluar cada modelo
for name, model_comp in models.items():
    print(f"\nModelo: {name}")
    if name == "Red Neuronal":
        #Entrenamiento específico para la red neuronal
        model_comp.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=0)

        #Realizar predicciones con la red neuronal
        nn_probs = model_comp.predict(X_test_scaled).flatten()  # predicciones probabilísticas
        nn_preds = (nn_probs > 0.5).astype(int)  # convertir a clases 0 o 1

        #Almacenar la precisión del modelo
        model_accuracies[name] = accuracy_score(y_test, nn_preds)

        print("Red Neuronal")
        print(classification_report(y_test, nn_preds))

    else:
        #Entrenamiento y predicción para Random Forest y Regresión Logística
        model_comp.fit(X_train_scaled, y_train)
        preds = model_comp.predict(X_test_scaled)

        #Calcular y mostrar el reporte de clasificación
        report = classification_report(y_test, preds, digits=2)
        print(report)

        #Almacenar la precisión del modelo
        model_accuracies[name] = accuracy_score(y_test, preds)

####Comparación de Precisión

In [ ]:
#Ordenar las precisiones de menor a mayor
sorted_accuracies = dict(sorted(model_accuracies.items(), key=lambda item: item[1]))

#Crear gráfico de barras para comparar precisiones
plt.figure(figsize=(8, 6))
bars = plt.bar(sorted_accuracies.keys(), sorted_accuracies.values(), color=['salmon', 'skyblue', 'lightgreen'])

#Agregar título y etiquetas
plt.title('Comparación de Precisión de Modelos', fontsize=16)
plt.xlabel('Modelos', fontsize=12)
plt.ylabel('Precisión', fontsize=12)

#Agregar las precisiones exactas sobre las barras
for bar, accuracy in zip(bars, sorted_accuracies.values()):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0, f'{accuracy:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

#Mostrar el gráfico
plt.show()

#### Comparación de F1-Score

In [ ]:
#Calcular y comparar F1-Score de los modelos
f1_scores = {}

#Evaluar F1-Score para cada modelo
for name, model_comp in models.items():
    model_comp.fit(X_train_scaled, y_train)

    #Obtener las predicciones del modelo
    preds_probs = model_comp.predict_proba(X_test_scaled) if hasattr(model_comp, 'predict_proba') else model_comp.predict(X_test_scaled)

    #Convertir probabilidades a predicciones binarias (0 o 1)
    if preds_probs.ndim > 1 and preds_probs.shape[1] > 1:
        preds = (preds_probs[:, 1] > 0.5).astype(int)
    else:
        preds = preds_probs.astype(int)

    f1 = f1_score(y_test, preds)
    f1_scores[name] = f1

#Agregar el modelo de Red Neuronal
nn_preds = model.predict(X_test_scaled)
nn_preds = (nn_preds > 0.5).astype(int)
f1_nn = f1_score(y_test, nn_preds)
f1_scores["Red Neuronal"] = f1_nn

#Ordenar los F1-Scores de menor a mayor
sorted_f1_scores = dict(sorted(f1_scores.items(), key=lambda item: item[1]))

#Gráfico de comparación de F1-Score
plt.figure(figsize=(8, 6))
bars = plt.bar(sorted_f1_scores.keys(), sorted_f1_scores.values(), color=['salmon', 'skyblue', 'lightgreen'])
plt.ylabel('F1-Score')
plt.title('Comparación de Modelos - F1 Score')
plt.ylim(0, 1)
for i, (bar, v) in enumerate(zip(bars, sorted_f1_scores.values())):
    plt.text(bar.get_x() + bar.get_width() / 2, v - 0.05, f"{v:.3f}", ha='center', fontweight='bold')
plt.show()

In [ ]:
#Encontrar el modelo con mejor F1-Score
mejor_modelo = max(f1_scores, key=f1_scores.get)
mejor_score = f1_scores[mejor_modelo]

#Generar el resumen interpretativo
print(f"El modelo con mejor desempeño fue **{mejor_modelo}**, obteniendo un F1-Score de {mejor_score:.3f}.")
print(f"Este modelo supera en rendimiento a los demás, basándose en un balance óptimo entre precisión y recall.")

## #11. Guardado de los Resultados con las Predicciones y Exportacion a GSC

### Google Cloud Storage

In [ ]:
#Guardar el dataset completo con predicciones
df_cleaned.to_csv('/content/dataset_con_predicciones.csv', index=False)
files.download('/content/dataset_con_predicciones.csv')

#Subir a Google Cloud Storage
blob = bucket.blob('dataset_con_predicciones.csv')
blob.upload_from_filename('/content/dataset_con_predicciones.csv')
print("Archivo 'dataset_con_predicciones.csv' subido a Google Cloud Storage.")

#Filtrar por tipo de vino y guardar archivos individuales
tipos_vino = df_cleaned['Tipo de Vino'].unique()

for tipo in tipos_vino:
    df_vino = df_cleaned[df_cleaned['Tipo de Vino'] == tipo]
    nombre_archivo = f'dataset_{tipo.lower()}.csv'
    ruta_local = f'/content/{nombre_archivo}'

    #Guardar CSV local
    df_vino.to_csv(ruta_local, index=False)

    #Descargar localmente en Colab
    files.download(ruta_local)

    #Subir a GCS
    blob = bucket.blob(nombre_archivo)
    blob.upload_from_filename(ruta_local)
    print(f"Archivo '{nombre_archivo}' subido a Google Cloud Storage.")

### Carga de Datos a Big Query

#### Validacion de Columnas

In [ ]:
#Renombrar las columnas para que cumplan con los requisitos de BigQuery
df_cleaned.columns = df_cleaned.columns.str.replace(' ', '_')  # Reemplaza espacios por guiones bajos
df_cleaned.columns = df_cleaned.columns.str.replace('/', '')  # Elimina slash
df_cleaned.columns = df_cleaned.columns.str.replace(')', '')  # Elimina paréntesis
df_cleaned.columns = df_cleaned.columns.str.replace('(', '')  # Elimina paréntesis

#Verificar los nuevos nombres de las columnas
print("Nombres de columnas después del renombrado:")
print(df_cleaned.columns)

#Crear la columna binaria
if 'Conformidad_Bin' not in df_cleaned.columns:
    df_cleaned['Conformidad_Bin'] = (df_cleaned['Conformidad'] == 'Sí').astype(int)

#Convertir 'Registro_muestra' a tipo datetime y luego a date
df_cleaned['Registro_muestra'] = pd.to_datetime(df_cleaned['Registro_muestra'], errors='coerce')
df_cleaned['Registro_muestra'] = df_cleaned['Registro_muestra'].dt.date

#Verificar los tipos de datos después de la conversión
print("\nDatos a importar en Google BigQuery:")
print(df_cleaned.dtypes)

#### Autenticación y Definición de la Tabla en BigQuery

In [ ]:
#Establecer la variable de entorno para la autenticación
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_path

#Crear cliente de BigQuery
client = bigquery.Client()
creds, project = default()
print("Autenticación exitosa con Google Cloud.")

# Definir proyecto, dataset y tabla en BigQuery
project_id = 'sistema-de-calidad-vinos' #Usar el ID del proyecto correcto
dataset_id = 'Dataset_Vinos_TFM' #Usar ID del connjunto de datos
table_name = 'Tabla_Vinos_Analisis'  #Nombre de la tabla en BigQuery (Editable)
table_id = f"{project_id}.{dataset_id}.{table_name}"

#Crear el dataset
datasets = list(client.list_datasets(project=project_id))
dataset_names = [d.dataset_id for d in datasets]

if dataset_id not in dataset_names:
    dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset_ref.location = "EU"  # Puedes cambiar la región si lo necesitas
    client.create_dataset(dataset_ref)
    print(f"Dataset '{dataset_id}' creado en el proyecto '{project_id}'.")

#Definir el esquema para BigQuery basado en las columnas del DataFrame
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("Tipo_de_Vino", "STRING"),
        bigquery.SchemaField("Tipo_de_Uva", "STRING"),
        bigquery.SchemaField("Registro_muestra", "DATE"),
        bigquery.SchemaField("Deposito", "STRING"),
        bigquery.SchemaField("Litros", "FLOAT"),
        bigquery.SchemaField("Grado_Alcoholico_Adquirido", "FLOAT"),
        bigquery.SchemaField("Sulfuroso_Libre_mgL", "FLOAT"),
        bigquery.SchemaField("Sulfuroso_Total_mgL", "FLOAT"),
        bigquery.SchemaField("Acidez_Total_gL", "FLOAT"),
        bigquery.SchemaField("Acidez_Volatil_gL", "FLOAT"),
        bigquery.SchemaField("pH", "FLOAT"),
        bigquery.SchemaField("Azucares_Reductores_gL", "FLOAT"),
        bigquery.SchemaField("Conformidad", "STRING"),
        bigquery.SchemaField("Conformidad_Bin", "INTEGER"),
    ],
    write_disposition="WRITE_APPEND",  #Si desea agregar datos a la tabla existente
)

#Cargar DataFrame completo
load_job = client.load_table_from_dataframe(df_cleaned, table_id, job_config=job_config)
load_job.result()
print(f" Datos cargados en la tabla general: {table_id}")

#### Conversión de Datos y Carga a BigQuery

In [ ]:
#Verificar los tipos de datos después de la conversión
print("Datos a importar en Google Cloud BigQuery:")
print(df_cleaned.dtypes)

#Cargar el DataFrame en BigQuery
tipos_vino = df_cleaned['Tipo_de_Vino'].unique()
for tipo in tipos_vino:
    df_vino = df_cleaned[df_cleaned['Tipo_de_Vino'] == tipo]
    table_name = f"Tabla_Vinos_{tipo.lower().replace(' ', '_')}"
    table_id = f"{project_id}.{dataset_id}.{table_name}"

    try:
    #Esperar a que el job termine
      load_job = client.load_table_from_dataframe(df_vino, table_id, job_config=job_config)
      load_job.result()
      print(f"Datos del tipo '{tipo}' cargados en: {table_id}")

    except Exception as e:
      print(f"Error al cargar datos del tipo '{tipo}': {e}")

In [ ]:
#Validar la cuenta de Google Cloud a donde los datos han sido cargados
creds, project = default()
print(f"El dataset ha sido subido al siguiente proyecto: {project}")
print(f"Cuenta autenticada: {creds.service_account_email}")